# Подготовка датасета

In [1]:
!pip install sentence-transformers -q

In [2]:
from tqdm import tqdm
import pandas as pd
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import torch

from sqlglot import parse_one
from sqlglot.diff import ChangeDistiller

from sqlalchemy import create_engine
from sklearn.utils import shuffle
from sqlalchemy import Connection

In [4]:
from abc import ABC
from dataclasses import dataclass

@dataclass
class Span(ABC):
    pass


@dataclass
class ExtendedSqlSpan(Span):
    NL : str
    sql_gold : str
    sql_pred : str
    df_soft : int
    df_flexible : int
    df_gold_IN_df_pred : bool
    df_pred_IN_df_gold : bool
    df_gold_columns : list[str]
    df_pred_columns : list[str]
    TED : int
    ERROR : str
    

### <div class='alert alert-info'>spans.py</div>


### <div class='alert alert-info'>general.py</div>


In [7]:
from dataclasses import dataclass
from abc import ABC
import numpy as np
from sentence_transformers import util
import pandas as pd
import zipfile
from sqlglot import exp
import sqlglot.optimizer
import re
# from spans import *
from pandas.testing import assert_frame_equal, assert_series_equal

def find_similar_sentences(sentence_model, target_sentence : str, sentences : list[str], count : int = 3):
    """
    Функция поиска похожих по смыслу предложений из набора `sentences` для указанного предложения `target_sentence`

    Parameters
    ----------
    sentence_model : Any
        Модель, позволяющая векторизовать текст
    target_sentence: str
        Предложение, для которого нужно найти похожие по смыслу предложения
    sentences : List[str]
        Набор предложений
    count : int
        Количество ожидаемых предложений
    """

    emb_target = sentence_model.encode(target_sentence)

    sims = []
    for i, sentence in enumerate(sentences):
        emb_sentence = sentence_model.encode(sentence)
        sim = util.pytorch_cos_sim(emb_sentence, emb_target)
        sims.append([i, np.float16(sim.squeeze())])

    nearest = sorted(sims, key=lambda pair : pair[1], reverse=True)
    similar_questions = [sentences[pair[0]] for pair in nearest if pair[1] != 1.0][:count]
    return similar_questions


def find_sql(text : str, start_keyword='SELECT'):
    """
    Функция, которая ищет в строке `text` первое вхождение самого длинного, правильного SQL запроса
    """

    matches = re.search(f'({start_keyword}).*', text, flags=re.IGNORECASE)
    if not matches:
        return ''

    begin_sql = matches.group()
    splitted = begin_sql.split()

    maybe_sql = ''
    last_success_pos = 0
    for i, word in enumerate(splitted):
        maybe_sql += f' {word}'
        try:
            sqlglot.transpile(maybe_sql)
            last_success_pos = i
        except:
            pass

    found_sql = ' '.join(splitted[:last_success_pos + 1])
    return found_sql


def table_similarity(dataframe1 : pd.DataFrame, dataframe2 : pd.DataFrame, mode : str) -> int:
    """
    Функция сравнения двух таблиц

    Parameters
    ----------
    dataframe1 : pd.DataFrame
        Первая таблица
    dataframe2 : pd.DataFrame
        Вторая таблица
    mode : str
        Режим сравнения. Допустимы режимы soft, strict, flexible
    """

    # if dataframe1.columns.shape != dataframe2.columns.shape:
    #     return False
    # if not (dataframe1.columns == dataframe2.columns).all():
    #     return False
    
    match mode:
        case 'soft':
            return int(dataframe1.sort_index().equals(dataframe2.sort_index()))
        case 'strict':
            return int(dataframe1.equals(dataframe2))
        case 'flexible':
            hash_1 = set(pd.util.hash_pandas_object(dataframe1, index=False))
            hash_2 = set(pd.util.hash_pandas_object(dataframe2, index=False))
            intersection = hash_1 & hash_2
            union = hash_1 | hash_2

            return len(intersection) / len(union) if len(union) != 0 else 1
        case _:
            raise Exception('Incorrect mode value')
     

def unzip_file(path, path_to):
    with zipfile.ZipFile(path, 'r') as zip_ref:
        zip_ref.extractall(path_to)


def schema_parse(sql : str, structure_dict : dict):
    """
    Функция, вытягивающая все названия таблиц и столбцов, которые упомянуты в запросе `sql`

    Parameters
    ----------
    sql : str
        SQL запрос
    table_structure : List[dict]
        Структура таблицы, которая может быть получена при помощи функции `structure_from_connection`
    """

    optimized_sql = sqlglot.optimizer.optimize(
        sqlglot.parse_one(sql),
        schema=structure_dict
    )

    buckets = {table.name : set([]) for table in optimized_sql.find_all(exp.Table)}
    for column in optimized_sql.find_all(exp.Column):
        table_of_col = column.table
        buckets[table_of_col].add(column.name)

    as_default = []
    for k, v in buckets.items():
        as_default.append({'table_name' : k, 'columns' : list(v)})

    return as_default


class ExcelIO(object):
    @staticmethod
    def write_spans(spans : list[Span], path : str):
        asdict = [span.__dict__ for span in spans]
        df = pd.DataFrame(asdict)
        df.to_excel(excel_writer=path, index=False)

    @staticmethod
    def read_excel(path : str):
        df = pd.read_excel(path)
        return df

def normalize_table(
    df: pd.DataFrame
) -> pd.DataFrame:
    """
    Normalizes a dataframe by:
    1. sorting columns in alphabetical order
    2. sorting rows using values from first column to last
    3. resetting index
    """
    # sort columns in alphabetical order
    sorted_df = df.reindex(sorted(df.columns), axis=1)
    # sort rows using values from first column to last
    sorted_df = sorted_df.sort_values(by=list(sorted_df.columns))
    # reset index
    sorted_df = sorted_df.reset_index(drop=True)
    return sorted_df

def subset_df(
    df_sub: pd.DataFrame,
    df_super: pd.DataFrame,
    verbose: bool = False,
) -> bool:
    """
    Checks if df_sub is a subset of df_super
    """
    if df_sub.empty:
        return True  # trivial case
    # make a copy of df_super so we don't modify the original while keeping track of matches
    df_super_temp = df_super.copy(deep=True)
    matched_columns = []
    for col_sub_name in df_sub.columns:
        col_match = False
        for col_super_name in df_super_temp.columns:
            col_sub = df_sub[col_sub_name].sort_values().reset_index(drop=True)
            col_super = (
                df_super_temp[col_super_name].sort_values().reset_index(drop=True)
            )
            try:
                assert_series_equal(
                    col_sub, col_super, check_dtype=False, check_names=False
                )
                col_match = True
                matched_columns.append(col_super_name)
                # remove col_super_name to prevent us from matching it again
                df_super_temp = df_super_temp.drop(columns=[col_super_name])
                break
            except AssertionError:
                continue
        if col_match == False:
            if verbose:
                print(f"no match for {col_sub_name}")
            return False
    df_sub_normalized = normalize_table(df_sub)

    # get matched columns from df_super, and rename them with columns from df_sub, then normalize
    df_super_matched = df_super[matched_columns].rename(
        columns=dict(zip(matched_columns, df_sub.columns))
    )
    df_super_matched = normalize_table(df_super_matched)

    try:
        assert_frame_equal(df_sub_normalized, df_super_matched, check_dtype=False)
        return True
    except AssertionError:
        return False

### <div class='alert alert-info'>dataset.py</div>


In [23]:
import string
import pandas as pd
from sqlalchemy import text, Connection, inspect

class IterableDataFrame:
    """
    Класс, позволяющий итерироваться в таблице типа `pd.DataFrame`
    """

    def __init__(self, df : pd.DataFrame):
        self.df = df
        self.__series = {}
        for idx in self.df.index:
            sample = {
                column : self.df[self.df.index == idx][column][idx] for column in self.df.keys()
            }
            self.__series[idx] = sample

    def __len__(self):
        return self.df.shape[0]

    def __as_list(self):
        return list(self.__series.values())
    
    def __iter__(self):
        return iter(self.__as_list())

    def __getitem__(self, index):
        return self.__as_list()[index]
    
    def at_index(self, index):
        return self.__series[index]


def tables_from_connection(conn : Connection):
    """
    Функция, возвращающая список названий всех таблиц для данного соединения `conn`

    Parameters
    ----------
    conn : sqlalchemy.Connection
        Соединение с базой данных
    """

    master = pd.DataFrame(conn.execute(text('SELECT * FROM sqlite_master')).fetchall())
    tables = list(master[master['type'] == 'table']['name'])
    return tables


def structure_from_connection(conn : Connection):
    """
    Функция, возвращающая список словарей вида {table_name, columns}, где table_name - str, а columns - List[str]

    Parameters
    ----------
    conn : sqlalchemy.Connection
        Соединение с базой данных
    """

    tables = tables_from_connection(conn)
    structure = []
    for table in tables:
        columns = pd.DataFrame(conn.execute(text(f'SELECT * FROM "{table}"')).fetchall()).columns.to_list()
        structure.append(
            {
                'table_name' : table,
                'columns' : columns
            })
        
    return structure


def structure_from_connection_dict(conn : Connection):
    """
    Функция, возвращающая словарь словарей вида {"Table" : {"Col" : "INT", ...}}

    Parameters
    ----------
    conn : sqlalchemy.Connection
        Соединение с базой данных
    """

    tables = tables_from_connection(conn)
    structure = {}
    for table in tables:
        columns = inspect(conn).get_columns(table)
        columns_meta = {column['name'] : column['type'] for column in columns}
        structure[table] = columns_meta

    return structure


def prepare_column_names(conn : Connection):
    """
    Функция, обрабатывающая базу данных из соединения `conn`. Функция переименовывает названия всех таблиц и их столбцов, 
    которые содержат whitespace и punctuation символы. Возвращает True, если переименовывание прошло успешно

    Parameters
    ----------
    conn : sqlalchemy.Connection
        Соединение с базой данных
    """
    
    structure = structure_from_connection(conn)
    for table in structure:
        for column in table['columns']:
            # new_name = str.lower(''.join([char for char in column if str.isalnum(char)]))
            new_name = str(''.join([char for char in column if str.isalnum(char)]))
            if new_name != column:
                conn.execute(text(
                    f'''ALTER TABLE "{table['table_name']}" RENAME COLUMN "{column}" TO "{new_name}"'''
                ))

        # new_table_name = str.lower(''.join([char for char in table['table_name'] if str.isalnum(char)]))
        new_table_name = str(''.join([char for char in table['table_name'] if str.isalnum(char)]))        
        if new_table_name != table['table_name']:
            conn.execute(text(f'''ALTER TABLE "{table['table_name']}" RENAME TO "{new_table_name}"'''))

    return True

### <div class='alert alert-info'>prompting.py</div>


In [9]:
import pandas as pd
import numpy as np
import sqlalchemy


class PromptBuilder:
    """
    Класс, отвечающий за создание промпта на основе указанных фичей
    """

    def __init__(self):
        self.__prompt = ''
        self.schema_linking = False


    def add_schema_linking(self, table_structure=None):
        """
        Метод, добавляющий режим использования фичи Schema Linking. 
        
        Parameters
        ----------
        table_structure : Any
            Структура таблицы, которая может быть получена с помощью функции `structure_from_connection`
        """

        self.table_structure = table_structure
        self.schema_linking = True
        return self


    def add_few_shot(self, queries, target_question : str, sentence_model):
        """
        Метод, отвечающий за добавление фичи Few-Shot в промпт

        Parameters
        ----------

        sentence_model : Any
            Модель, позволяющая векторизовать текст
        target_question : str
            Вопрос, для которого нужно найти похожие по смыслу вопросы
        queries : Any
            Набор вопросов и запросов, среди которых нужно найти ближайшие по смыслу вопросы. Объект должен являться матрицей Nx2
        """

        questions = [sample['question'] for sample in queries]

        input_examples = []
        similar = find_similar_sentences(sentence_model, target_question, questions, count=3)
        for sample in queries:
            curr_qs = sample['question']
            if curr_qs in similar:
                input_examples.append([curr_qs, sample['query']])

        few_shot_template = ''
        for ex in input_examples:
            few_shot_template += f'Q: {ex[0]}\n'
            few_shot_template += f'A: {ex[1]}\n'

        self.__prompt += few_shot_template + '\n'
        return self
    

    def add_schema_template(self, db_conn : sqlalchemy.Connection):
        """
        Метод, отвечающий за добавление фичи Schema Template в промпт

        Parameters
        ----------
        db_conn : sqlalchemy.Connection
            Соединение с базой данных
        """

        if self.schema_linking:
            structure = self.table_structure
        else:
            structure = structure_from_connection(db_conn)

        schema_template = ''
        for table in structure:
            schema_template += f"{table['table_name']}({', '.join(table['columns'])});\n"

        self.__prompt += schema_template + '\n'
        return self


    def add_cell_value_referencing(self, db_conn : sqlalchemy.Connection, count=1):
        """
        Метод, отвечающий за добавление фичи Cell Value Referencing в промпт

        Parameters
        ----------
        db_conn : sqlalchemy.Connection
            Соединение с базой данных
        count : int
            Ожидаемое количество примеров для добавления. По умолчанию равно 1
        """

        if self.schema_linking:
            tables = [table['table_name'] for table in self.table_structure]
        else:
            tables = tables_from_connection(db_conn)

        data_information = []
        for table in tables:
            if self.schema_linking:
                instance = [bucket for bucket in self.table_structure if bucket['table_name'] == table][0]
                pd_table = pd.read_sql(f'SELECT * FROM {table}', db_conn)[instance['columns']]
            else:
                pd_table = pd.read_sql(f'SELECT * FROM {table}', db_conn)
            
            indexes = np.random.randint(0, pd_table.shape[0], size=count)
            series = [pd_table[pd_table.index == idx].to_numpy() for idx in indexes]

            data_information.append({
                'table_name' : table,
                'examples' : [f"[{', '.join(map(str,list(ser.reshape(ser.shape[1]))))}]" for ser in series]
            })

        value_template = ''
        for data in data_information:
            value_template += f"{data['table_name']}({', '.join(data['examples'])});\n"

        self.__prompt += value_template + '\n'
        return self


    def add_message(self, message : str):
        self.__prompt += message + '\n'
        return self


    def build_prompt(self):
        return self.__prompt

### <div class='alert alert-info'>models-evaluation.ipynb</div>


In [24]:
import shutil
from sqlalchemy import create_engine

source_db = '/kaggle/input/main-database/main_database.sqlite'
dest_db = '/kaggle/working/main_database_copy.sqlite'

shutil.copyfile(source_db, dest_db)

engine = create_engine(f'sqlite:///{dest_db}', echo=False)
conn = engine.connect()

print(prepare_column_names(conn))
structure_from_connection(conn)

True


[{'table_name': 'Остатки2024',
  'columns': ['Артикул',
   'Номенклатура',
   'Ед',
   '01042024',
   '02042024',
   '03042024',
   '04042024',
   '05042024',
   '06042024',
   '07042024',
   '08042024',
   '09042024',
   '10042024',
   '11042024',
   '12042024',
   '13042024',
   '14042024',
   '15042024',
   '16042024',
   '17042024',
   '18042024',
   '19042024',
   '20042024',
   '21042024',
   '22042024',
   '23042024',
   '24042024',
   '25042024',
   '26042024',
   '27042024',
   '28042024',
   '29042024',
   '30042024',
   'Итого']},
 {'table_name': 'Остатки2023',
  'columns': ['Артикул',
   'Номенклатура',
   'Ед',
   '01042023',
   '02042023',
   '03042023',
   '04042023',
   '05042023',
   '06042023',
   '07042023',
   '08042023',
   '09042023',
   '10042023',
   '11042023',
   '12042023',
   '13042023',
   '14042023',
   '15042023',
   '16042023',
   '17042023',
   '18042023',
   '19042023',
   '20042023',
   '21042023',
   '22042023',
   '23042023',
   '24042023',
   '2504

In [11]:
queries = IterableDataFrame(pd.read_excel('/kaggle/input/main-database1/NLSQL.xlsx'))

# Препроцессинг промпта

In [12]:
sentence_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
class HuggingFaceModelInference:
    def __init__(self, path):
        self.path = path
        self.evaluated = False
        self.is_downloaded = False


    def __load_model(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.path, trust_remote_code=True)
        self.model = AutoModelForCausalLM.from_pretrained(
                    self.path,
                    torch_dtype=torch.float16,
                    device_map="auto",
                    max_memory={0: "10GiB", 1: "10GiB"},  
                    offload_folder="./offload", 
                    trust_remote_code=True
                    )

    def __inference(self, prompt):
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        with torch.inference_mode():  
            inputs = self.tokenizer(prompt, return_tensors="pt").to(self.model.device) 
            generate_ids = self.model.generate(
                            **inputs,
                            max_length=2048,
                            num_return_sequences=1,
                            temperature=0.2, 
                            top_p=0.95,
                            do_sample=True,
                            use_cache=True 
                            )
    
            output = self.tokenizer.decode(
                    generate_ids[0, inputs.input_ids.shape[1]:],
                    skip_special_tokens=True
                    )
            
        return output
    

    def evaluate(self, queries : IterableDataFrame, connection : Connection):
        if not self.is_downloaded:
            self.__load_model()
            self.is_downloaded = True

        self.model.eval()

        logger : list[ExtendedSqlSpan] = []
        summary = 0
        for query in tqdm(queries):
            question = query['question']
            gold_sql = query['query']

            prompt = PromptBuilder()\
                .add_message('### You are an expert SQL developer with deep knowledge of database optimization, correct syntax, and efficient query design. Your task is to generate accurate, performant SQL queries based on the provided input.')\
                .add_message("### Table schema:")\
                .add_schema_template(conn)\
                .add_message("### Examples of data")\
                .add_cell_value_referencing(conn, count=1)\
                .add_message(f"### Your task: {question}")\
                .build_prompt()
            

            output = self.__inference(prompt)
            pred_sql = find_sql(output, start_keyword='SELECT')
            
            df_gold = pd.read_sql(gold_sql, connection)
            query_err = 'OK'
            try:
                df_pred = pd.read_sql(pred_sql, connection)
                
                span_df_soft = table_similarity(df_pred, df_gold, mode='soft')
                span_df_flexible = table_similarity(df_pred, df_gold, mode='flexible')
                span_gold_IN_pred = subset_df(df_gold, df_pred)  # TODO: Добавить проверку
                span_pred_IN_gold = subset_df(df_pred, df_gold)  # TODO: Добавить проверку
                span_pred_columns = df_pred.columns.to_list()
                span_ted = self.__ted_compare(pred_sql, gold_sql)
                
            except Exception as e:
                error_type = type(e).__name__
                error_details = str(e)
                query_err = error_details
                print(f"Ошибка при выполнении SQL или сравнении результатов: {error_type} - {error_details}")
                print(f"Проблемный SQL запрос: {pred_sql}")
                
                if "table_similarity" in error_details:
                    print("Проблема в функции сравнения таблиц")
                    if not df_pred.columns.equals(df_gold.columns):
                        print("Колонки не совпадают:")
                        print(f"Предсказанные: {df_pred.columns.to_list()}")
                        print(f"Эталонные: {df_gold.columns.to_list()}")
                
                span_df_soft = 0.0
                span_df_flexible = 0.0
                span_gold_IN_pred = subset_df(df_gold, df_pred)  # TODO: Добавить проверку
                span_pred_IN_gold = subset_df(df_pred, df_gold)
                span_pred_columns = ['УВЫ']
                span_ted = self.__ted_compare(pred_sql, gold_sql)
                # Можно также записать ошибку в логгер для последующего анализа
                # self.error_log.append({
                #     'question': question,
                #     'pred_sql': pred_sql,
                #     'gold_sql': gold_sql,
                #     'error_type': error_type,
                #     'error_details': error_details
                # })
            sql_span = ExtendedSqlSpan(
                    NL                 =question,
                    sql_gold           =gold_sql,
                    sql_pred           =pred_sql,
                    df_soft            =span_df_soft,
                    df_flexible        =span_df_flexible,
                    df_pred_IN_df_gold =span_pred_IN_gold,
                    df_gold_IN_df_pred =span_gold_IN_pred,
                    df_gold_columns    =df_gold.columns.to_list(),
                    df_pred_columns    =span_pred_columns,
                    TED                =span_ted,
                    ERROR              =query_err
                )
            summary += span_df_flexible
            logger.append(sql_span) 
        
        self.summary = summary
        self.queries_count = len(queries)
        self.logger = logger
        self.evaluated = True


    def accuracy(self):
        """
        Значение метрики Accuracy для последнего запуска модели
        """

        if not self.evaluated:
            raise Exception('Model was not been evaluated')
        
        return self.summary / self.queries_count
    

    def __ted_compare(self, sql1 : str, sql2 : str):
        """
        Компоратор для двух деревьев
        """
        
        try:
            exp1 = parse_one(sql1)
            exp2 = parse_one(sql2)
        except:
            return .0

        distiller = ChangeDistiller()
        _ = distiller.diff(exp1, exp2)
        return distiller._dice_coefficient(exp1, exp2)


    def TED(self):
        """
        Значение метрики Tree Edit Distance для последнего запуска модели
        """

        if not self.evaluated:
            raise Exception('Model was not been evaluated')
        
        summary = 0
        for span in self.logger:
            summary += self.__ted_compare(span.sql_pred, span.sql_gold)

        return summary / self.queries_count

## 1. SQLCoder 7b

In [25]:
sqlcoder = HuggingFaceModelInference('defog/sqlcoder-7b-2')
sqlcoder.evaluate(shuffle(queries), conn)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

 19%|█▊        | 8/43 [01:16<05:50, 10.02s/it]

Ошибка при выполнении SQL или сравнении результатов: OperationalError - (sqlite3.OperationalError) near "ilike": syntax error
[SQL: SELECT * FROM Остатки2024 WHERE Номенклатура ilike '%нагреватель%';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Проблемный SQL запрос: SELECT * FROM Остатки2024 WHERE Номенклатура ilike '%нагреватель%';


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Ошибка при выполнении SQL или сравнении результатов: OperationalError - (sqlite3.OperationalError) no such column: Количество
[SQL: SELECT * FROM Остатки2024 WHERE Количество = 3;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Проблемный SQL запрос: SELECT * FROM Остатки2024 WHERE Количество = 3;


...

 98%|█████████▊| 42/43 [06:26<00:05,  5.25s/it]

Ошибка при выполнении SQL или сравнении результатов: OperationalError - (sqlite3.OperationalError) no such column: p.СуммаНДС
[SQL: SELECT * FROM Продажи p WHERE p.СуммаНДС = 613.33;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Проблемный SQL запрос: SELECT * FROM Продажи p WHERE p.СуммаНДС = 613.33;


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Ошибка при выполнении SQL или сравнении результатов: OperationalError - (sqlite3.OperationalError) no such column: o.Склад
[SQL: SELECT * FROM Остатки2024 o WHERE o.Склад IS NULL;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Проблемный SQL запрос: SELECT * FROM Остатки2024 o WHERE o.Склад IS NULL;


100%|██████████| 43/43 [06:30<00:00,  9.09s/it]


In [26]:
ExcelIO.write_spans(sqlcoder.logger, 'out.xlsx')
sqlcoder.accuracy(), sqlcoder.TED()

(0.5878477589280333, 0.6617285258037561)

In [60]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()

In [14]:
#sqlcoder.accuracy()

In [15]:
#sqlcoder.logger

## DeepSeek 6.7b

In [16]:
#deepseek = HuggingFaceModelInference('deepseek-ai/deepseek-coder-6.7b-instruct')
#deepseek.evaluate(shuffle(queries.as_list())[:10], conn) 

In [17]:
#torch.cuda.empty_cache()

## 3. Chat2DB 7b

In [18]:
# chat2db = HuggingFaceModelInference('Chat2DB/Chat2DB-SQL-7B')
# chat2db.evaluate(shuffle(dataset)[:20])

In [19]:
# chat2db.accuracy(), chat2db.sql_similarity(), np.mean(chat2db.exec_time)

In [20]:
# dump_inference('Chat2DB-SQL-7B', chat2db.exec_time, chat2db.sql_similarity(), chat2db.accuracy())

## 5. DuckDB-NSQL 7b

In [21]:
# duckdb = HuggingFaceModelInference('motherduckdb/DuckDB-NSQL-7B-v0.1')

In [22]:
# duckdb.evaluate(shuffle(dataset)[:30])

In [23]:
# duckdb.accuracy(), duckdb.sql_similarity(), np.mean(duckdb.exec_time)

In [24]:
# dump_inference('DuckDB-NSQL-7B-v0.1', duckdb.exec_time, duckdb.sql_similarity(), duckdb.accuracy())

## Прочее

In [25]:
from numba import cuda
import gc
#cuda.devices.gpus[0].reset()
#cuda.devices.gpus[1].reset()
#gc.collect()